In [ ]:
!pip install pubchempy
!pip install rdkit
!pip install torch_geometric

  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=6d7571cc175e457ff2c82f6166137af4855560c58db4c39a79f88ff30ebdf306
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00


In [ ]:
from pandas.plotting import table
from rdkit import Chem
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import pubchempy
import rdkit
import time
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset = MoleculeNet(root=".", name="lipo")
data = dataset[0]

Processing...
Done!


# EDA

In [ ]:
print(dataset[0])
print("Number of atoms in the first compound of the dataset: " + str(dataset[0].num_nodes))
print("Matrix containing the features of each atom of the first compound of the dataset:")
print(dataset[0].x)
print("Number of edges connecting the atoms of the first compound: " + str(dataset[0].num_edges))
print("Adjacency list representing the edges connecting the atoms of the first compound:")
print(dataset[0].edge_index)
print("Lipophilocity value for the first compound of the dataset: " + str(dataset[0].y))

Data(x=[24, 9], edge_index=[2, 54], edge_attr=[54, 3], smiles='Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14', y=[1, 1])
Number of atoms in the first compound of the dataset: 24
Matrix containing the features of each atom of the first compound of the dataset:
tensor([[ 6,  0,  4,  5,  3,  0,  4,  0,  0],
        [ 7,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  0],
        [ 7,  0,  3,  5,  0,  0,  4,  0,  1],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  1],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  1],
        [ 7,  0,  3,  5,  0,  0,  3,  0,  1],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  1],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [17,  0,  1,  5,  0,  0,  4,  0,  0],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1],
        [ 

# Model
**Input:** The model described below takes as input a graph where node information is represented as feature vectors.

**Architecture:** This model is composed of 4 consecutive **graph convolutional layers** which are defined as: update($h_{t-1}$, aggregate($\sum_{v_i \in N_{v_t}}v_i)$), where **aggregate** on a given node $v_t$ is defined as $\sum_{v_i \in N_{v_t}}(\frac{W*v_i}{degree(v_t)})$, and **update** is defined as the nonlinear tanh function.

* Note that Graph Convolutional operations are **permutation equivariant**

**Output:** At the end of the network we use both a **global avg pooling (GAP)** and a **global max pooling (GMP)**, concatenate them and feed them to a lienar layer which outputs a regression value.

* Note that **GAP and GMP** are both **permutation invariant** operations

In [ ]:
embedding_size = 64
class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)

        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index),
                            gap(hidden, batch_index)], dim=1)

        # Classifier (Linear).
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Number of parameters:  13249


# TRAIN/TEST DATALOADER


In [ ]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Data loader
data_size = len(dataset)
NUM_GRAPHS_PER_BATCH = 64
NUM_EPOCHS = 900

torch.manual_seed(12345)

#randomize and split the data
dataset = dataset.shuffle()

train_dataset = dataset[:int(data_size * 0.8)]
test_dataset = dataset[int(data_size * 0.8):]

loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

print('\n======== data distribution =======\n')
print("Size of training data: {} graphs".format(len(train_dataset)))
print("Size of testing data: {} graphs".format(len(test_dataset)))


======== data distribution =======

Size of training data: 3360 graphs
Size of testing data: 840 graphs


# TRAINING

In [ ]:
# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)

# Calculate accuracy r2
def r2_accuracy(pred_y, y):
    score = r2_score(y, pred_y)
    return round(score, 2)*100

# Data generated
embeddings = []
losses = []
accuracies = []
outputs = []
targets = []

# Use GPU for training, if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def train(data):
    # Enumerate over the data
    for batch in loader:
      # Use GPU
      batch.to(device)
      # Reset gradients
      optimizer.zero_grad()
      # Passing the node features and the connection info
      pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
      # Calculating the loss and gradients
      loss = loss_fn(pred, batch.y)
      acc = r2_accuracy(pred.cpu().detach().numpy(), batch.cpu().y.detach().numpy())

      loss.backward()
      # Update using the gradients
      optimizer.step()
    return loss, acc, pred, batch.y, embedding

print('\n======== Starting training ... =======\n')
start_time = time.time()

losses = []
for epoch in range(NUM_EPOCHS):
    loss, acc, pred, target, h = train(data)
    losses.append(loss)
    accuracies.append(acc)
    outputs.append(pred)
    targets.append(target)

    if epoch % 100 == 0:
      # print(f"Epoch {epoch} | Train Loss {loss}")
      print(f'Epoch {epoch:>3} | Loss: {loss:.5f} | Acc: {acc:.2f}%')

print("\nTraining done!\n")
elapsed = time.time() - start_time
minutes_e = elapsed//60
print("--- training took:  %s minutes ---" % (minutes_e))


======== Starting training ... =======

Epoch   0 | Loss: 1.21428 | Acc: -1.00%
Epoch 100 | Loss: 0.57189 | Acc: 62.00%
Epoch 200 | Loss: 0.31194 | Acc: 70.00%
Epoch 300 | Loss: 0.21917 | Acc: 86.00%
Epoch 400 | Loss: 0.16295 | Acc: 89.00%
Epoch 500 | Loss: 0.22088 | Acc: 85.00%
Epoch 600 | Loss: 0.27331 | Acc: 85.00%
Epoch 700 | Loss: 0.09960 | Acc: 92.00%
Epoch 800 | Loss: 0.10423 | Acc: 90.00%

Training done!

--- training took:  9.0 minutes ---


In [ ]:
import pandas as pd

# One batch prediction
test_batch = next(iter(test_loader))
with torch.no_grad():
    test_batch.to(device)
    pred, embed = model(test_batch.x.float(), test_batch.edge_index, test_batch.batch)
    df = pd.DataFrame()
    df["y"] = test_batch.y.tolist()
    df["y_pred"] = pred.tolist()
df["real"] = df["y"].apply(lambda row: row[0])
df["pred"] = df["y_pred"].apply(lambda row: row[0])
df.head()

test_acc = r2_accuracy(df["real"], df["pred"])

print("Test accuracy is {}%".format(round(test_acc, 2) ))

Test accuracy is 32.0%
